In [19]:
!python main.py experiments/general.cfg problems/hhcrsp/hhcrsp_5_4.cfg variants/hhcrsp.cfg -v

runNumber: 0
population:
	(2.96, 2.18, 2.72) = 7.85432429889
	(1.58, 1.57, 1.83) = 4.97805890481
	(1.94, 1.22, 3.00) = 6.16347931701
	(1.81, 2.22, 2.07) = 6.10166312549
	(1.02, 2.78, 1.27) = 5.07171602221
	(2.17, 1.10, 2.52) = 5.79246231111
	(1.28, 1.87, 2.88) = 6.02596272505
	(2.08, 1.32, 2.11) = 5.5157346222
	(1.61, 1.65, 1.04) = 4.30009142836
	(1.84, 2.94, 2.50) = 7.27897226564
('--> tree', [(0,), (1,), (2,), (0, 1)])
--->>> Schedule --------------------------
[(3, 0.2710586776435733)]
[(1, 0.08293104307752186), (2, 0.7788278109579658)]
------------------------------------------

0 {'success': 1, 'LS_evaluations': 0, 'minSubProblem': 0, 'LS_iterations': 0, 'bestFitness': '(2.08, 2.78, 1.27) = 9.5', 'evaluations': 1}
